Convert CSV with bounding box information to YOLO OBB format

In [ ]:
import pandas as pd
import cv2
import math
import os
import shutil

output_path = './yolo_single_class'
train_file_path = 'train_annotations.csv'
val_file_path = 'val_annotations.csv'
test_file_path = 'test_annotations.csv'
conditions = ['Cavitated', 'Retained Root', 'Crowned',
              'Filled', 'Impacted', 'Implant']
treatments = ['Filling', 'Root Canal', 'Extraction', 'None']
simple = ['Tooth']
from_diagnosis = {
    'Cavitated': simple[0],
    'Retained Root': simple[0],
    'Crowned': simple[0],
    'Filled': simple[0],
    'Impacted': simple[0],
    'Implant': simple[0],
}

In [ ]:
def createYOLODirectories():
    if not os.path.exists(output_path + '/images'):
        os.makedirs(output_path + '/images')
    if not os.path.exists(output_path + '/labels'):
        os.makedirs(output_path + '/labels')
    if not os.path.exists(output_path + '/images' + '/train'):
        os.makedirs(output_path + '/images' + '/train')
    if not os.path.exists(output_path + '/images' + '/test'):
        os.makedirs(output_path + '/images' + '/test')
    if not os.path.exists(output_path + '/images' + '/val'):
        os.makedirs(output_path + '/images' + '/val')
    if not os.path.exists(output_path + '/labels' + '/train'):
        os.makedirs(output_path + '/labels' + '/train')
    if not os.path.exists(output_path + '/labels' + '/test'):
        os.makedirs(output_path + '/labels' + '/test')
    if not os.path.exists(output_path + '/labels' + '/val'):
        os.makedirs(output_path + '/labels' + '/val')
createYOLODirectories()

In [ ]:
def createOrAppendToTxtFile(txtFileName, line):
    line = ' '.join([str(x) for x in line]) + '\n'
    if not os.path.exists(txtFileName):
        with open(txtFileName, 'w') as file:
            file.write(line)
    else:
        with open(txtFileName, 'a') as file:
            file.write(line)



def get_rotated_corner_points(box):
    """Get the corner points of a rotated bounding box.
    Note: It's important to take into account
    the original width and height of the image to get the correct coordinates.

    Args:
        box (dict): The bounding box with the following keys:
            - x: The x-coordinate of the top left corner (in percentage, 0-100)
            - y: The y-coordinate of the top left corner (in percentage, 0-100)
            - width: The width of the box (in percentage, 0-100)
            - height: The height of the box (in percentage, 0-100)
            - rotation: The rotation angle in degrees (optional)
            - original_width: The original width of the image (in pixels)
            - original_height: The original height of the image  (in pixels)
    Returns (list): The normalized corner points [(x1, y1), (x2, y2), (x3, y3), (x4, y4)]
    """
    image_width, image_height = box["original_width"], box["original_height"]
    w, h = box["width"] * image_width / 100, box["height"] * image_height / 100
    a = math.pi * (box["rotation"] / 180.0) if "rotation" in box else 0.0
    cos_a, sin_a = math.cos(a), math.sin(a)

    x1, y1 = box["x"] * image_width / \
        100, box["y"] * image_height / 100  # top left
    x2, y2 = x1 + w * cos_a, y1 + w * sin_a  # top right
    x3, y3 = x2 - h * sin_a, y2 + h * cos_a  # bottom right
    x4, y4 = x1 - h * sin_a, y1 + h * cos_a  # bottom left

    coords = [(x1, y1), (x2, y2), (x3, y3), (x4, y4)]
    normalized_coords = [(coord[0] / image_width,  coord[1] / image_height) for coord in coords]
    return normalized_coords
    

def BndBox2YoloLine(img, xmin, ymin, width, height, degree, classIndex, original_width, original_height):
    box = {
        "x": xmin,
        "y": ymin,
        "width": width,
        "height": height,
        "rotation": degree,
        "original_width": original_width,
        "original_height": original_height
    }
    corners = get_rotated_corner_points(box)
    x1, y1 = corners[0]
    x2, y2 = corners[1]
    x3, y3 = corners[2]
    x4, y4 = corners[3]
    return [classIndex, x1, y1, x2, y2, x3, y3, x4, y4]

In [ ]:
def toOBBYOLO(data, type = 'train', target = 'condition'):
    for index, row in data.iterrows():
        condition = row['condition']
        treatment = row['treatment']
        if target == 'condition':
            classIndex = conditions.index(condition)
        elif target == 'treatment':
            classIndex = treatments.index(treatment)
        else:
            classIndex = simple.index(from_diagnosis[condition])
        image = row['image']
        xmin = row['x']
        ymin = row['y']
        width = row['width']
        height = row['height']
        degree = row['rotation']
        img_width = row['image_width']
        img_height = row['image_height']
        if 'jpg' in image:
            txt_file_path = image.split('.jpg')[0] + '.txt'
        elif 'jpeg' in image:
            txt_file_path = image.split('.jpeg')[0] + '.txt'
        else:
            image = image + '.jpg'
            txt_file_path = image.split('.jpg')[0] + '.txt'
        txt_file_path = output_path  + '/labels/' + type + '/' + txt_file_path # change the path to the respective folder
        yolo_line = BndBox2YoloLine(
            image, xmin, ymin, width, height, degree, classIndex, img_width, img_height)
        createOrAppendToTxtFile(txt_file_path, yolo_line)
        img_output_path = output_path  + '/images/' + type + '/' + image
        if not os.path.exists(img_output_path):
            img = cv2.imread(image.split('_')[0] + '/' + image)
            cv2.imwrite(img_output_path, img)              
        

In [ ]:
train = pd.read_csv(train_file_path)
val = pd.read_csv(val_file_path)
test = pd.read_csv(test_file_path)
print(train.shape, val.shape, test.shape)
print(train.shape[0] + val.shape[0] + test.shape[0])

In [ ]:
toOBBYOLO(train, 'train', 'simple')
toOBBYOLO(val, 'val', 'simple')
toOBBYOLO(test, 'test', 'simple')

In [ ]:
shutil.copy('yolo_single_class.yaml', output_path) # move yaml file to output directory
shutil.make_archive(output_path, 'zip', output_path) # zip the output directory